# CNN1D Model — Emission Point Classification with Random Search and Cross-Validation

This notebook develops a **1D Convolutional Neural Network (CNN)** to classify particle emission sources (E1, E2, E3) using flattened sensor windows as input.

Unlike the MLP baseline, this model leverages **local temporal dependencies** across sensor readings using convolutional filters. To ensure robust evaluation and performance generalization, we apply:

- **Random Search** over a defined space of hyperparameters;
- **K-Fold Cross-Validation (K=5)** for fair performance estimation;
- Final evaluation with test data using the best discovered configuration.

---

## Notebook Structure

0. Import tensorflow and keras
1. Check available GPU devices
2. Confira CUDA support
3. Print tensorflow version
4. Check python version
5. Import required libraries
6. Load and preprocess dataset
7. Define hyperparams search space
8. Perform random search with K-fold Cross-Validation
9. Select best hyperparameter configuration
10. Final train/test split and reshape
11. Build and train final model
12. Evaluate final model on test set
13. Save final model

---

> ✅ This notebook introduces a more expressive model class than MLP by incorporating temporal patterns via CNN1D.  
> ⚙️ Combined with random search and cross-validation, this method offers a balance between speed and generalization for emission classification based on simulated sensor data.


## 0. Import TensorFlow and Keras

We begin by importing TensorFlow and the Keras API, which will be used to build and train the CNN1D model.


In [1]:
import tensorflow as tf
from tensorflow import keras

## 1. Check Available GPU Devices

We verify whether a GPU is available for model training. This ensures that the model benefits from hardware acceleration.


In [2]:
print("Num of GPUs avaliable: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num of GPUs avaliable:  1


## 2. Confirm CUDA Support

This step checks if TensorFlow was built with CUDA GPU support, confirming compatibility for training on GPU.


In [3]:
tf.test.is_built_with_cuda()

True

## 3. Print TensorFlow Version

We print the TensorFlow version to ensure reproducibility and compatibility.


In [4]:
print(tf.version.VERSION)

2.10.0


## 4. Check Python Version

Verifying the Python version can help ensure compatibility with the notebook environment.


In [5]:
import sys
sys.version

'3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:16:53) [MSC v.1929 64 bit (AMD64)]'

## 5. Import Required Libraries

We import all necessary libraries for data manipulation (NumPy, pandas), model building (Keras), evaluation, and preprocessing (scikit-learn).


In [6]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

## 6. Load and Preprocess Dataset

We load the dataset from the processed directory and:
- Remove the target column `Emission_Point` from features;
- Convert all features to numeric types and fill any missing values;
- Normalize the features using `StandardScaler`.


In [7]:
# Load dataset
df = pd.read_csv("../data/processed/complete_dataset.csv")
X = df.drop(columns=["Emission_Point"])
y = df["Emission_Point"]

# Convert to numeric and fill missing
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Final shape for all inputs
X_scaled = np.array(X_scaled)
y_cat = np.array(y_cat)

## 7. Define Hyperparameter Search Space

We define the space for the random search over 3 hyperparameters:
- `filters` (number of convolution filters),
- `kernel` (size of the convolution window),
- `dropout` (regularization strength).

We also initialize the `KFold` strategy for 5-fold cross-validation.


In [8]:
# Define search space
param_space = {
    "filters": [32, 64, 128],
    "kernel": [3, 5, 7],
    "dropout": [0.2, 0.3, 0.4]
}
n_samples = 5  # Number of random configs to try
kf = KFold(n_splits=5, shuffle=True, random_state=42)
random_results = []


## 8. Perform Random Search with K-Fold Cross-Validation

For each sampled configuration:
- A CNN1D is built with the specified hyperparameters;
- The model is trained and validated across 5 folds;
- The mean validation accuracy is stored for later comparison.


In [9]:
for i in range(n_samples):
    params = {
        "filters": random.choice(param_space["filters"]),
        "kernel": random.choice(param_space["kernel"]),
        "dropout": random.choice(param_space["dropout"])
    }

    print(f"\n🔧 Testing configuration {i+1}/{n_samples}: {params}")
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_cat[train_idx], y_cat[val_idx]

        # Reshape for CNN1D
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        clear_session()
        model = Sequential([
            Input(shape=(X_train.shape[1], 1)),
            Conv1D(params["filters"], kernel_size=params["kernel"], activation='relu'),
            BatchNormalization(),
            MaxPooling1D(pool_size=2),
            Dropout(params["dropout"]),
            Flatten(),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(3, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(X_train, y_train,
                  validation_data=(X_val, y_val),
                  epochs=5,
                  batch_size=32,
                  verbose=0,
                  callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

        _, acc = model.evaluate(X_val, y_val, verbose=0)
        fold_accuracies.append(acc)

    avg_acc = np.mean(fold_accuracies)
    print(f"✅ Avg Accuracy: {avg_acc:.4f}")
    random_results.append((params, avg_acc))



🔧 Testing configuration 1/5: {'filters': 64, 'kernel': 3, 'dropout': 0.2}
✅ Avg Accuracy: 0.8942

🔧 Testing configuration 2/5: {'filters': 64, 'kernel': 7, 'dropout': 0.3}
✅ Avg Accuracy: 0.8935

🔧 Testing configuration 3/5: {'filters': 128, 'kernel': 7, 'dropout': 0.2}
✅ Avg Accuracy: 0.8935

🔧 Testing configuration 4/5: {'filters': 128, 'kernel': 3, 'dropout': 0.3}
✅ Avg Accuracy: 0.8916

🔧 Testing configuration 5/5: {'filters': 64, 'kernel': 5, 'dropout': 0.4}
✅ Avg Accuracy: 0.8914


## 9. Select Best Hyperparameter Configuration

We identify and print the best-performing hyperparameter configuration based on average validation accuracy across folds.


In [10]:
best_config = sorted(random_results, key=lambda x: x[1], reverse=True)[0]
print(f"\n🏆 Best config: {best_config[0]}")
print(f"📈 Best Validation Accuracy: {best_config[1]:.4f}")



🏆 Best config: {'filters': 64, 'kernel': 3, 'dropout': 0.2}
📈 Best Validation Accuracy: 0.8942


## 10. Final Train/Test Split and Reshape

The entire dataset is split into 80% training and 20% testing.  
Features are reshaped to match the CNN1D input format `(samples, time steps, channels)`.


In [11]:
# Split full dataset for final evaluation
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_scaled, y_cat, test_size=0.2, stratify=y_cat, random_state=42)

X_train_final = X_train_final.reshape(X_train_final.shape[0], X_train_final.shape[1], 1)
X_test_final = X_test_final.reshape(X_test_final.shape[0], X_test_final.shape[1], 1)

clear_session()
model = Sequential([
    Input(shape=(X_train_final.shape[1], 1)),
    Conv1D(best_config[0]["filters"], kernel_size=best_config[0]["kernel"], activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(best_config[0]["dropout"]),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_final, y_train_final,
                    validation_data=(X_test_final, y_test_final),
                    epochs=20, batch_size=32,
                    callbacks=[EarlyStopping(patience=7, restore_best_weights=True)])


Epoch 1/20
5873/5873 [==============================] - 26s 4ms/step - loss: 0.3783 - accuracy: 0.8363 - val_loss: 0.2531 - val_accuracy: 0.8873
Epoch 2/20
5873/5873 [==============================] - 27s 5ms/step - loss: 0.2890 - accuracy: 0.8654 - val_loss: 0.2311 - val_accuracy: 0.8867
Epoch 3/20
5873/5873 [==============================] - 27s 5ms/step - loss: 0.2683 - accuracy: 0.8715 - val_loss: 0.2231 - val_accuracy: 0.8867
Epoch 4/20
5873/5873 [==============================] - 26s 4ms/step - loss: 0.2619 - accuracy: 0.8739 - val_loss: 0.2087 - val_accuracy: 0.8915
Epoch 5/20
5873/5873 [==============================] - 26s 4ms/step - loss: 0.2495 - accuracy: 0.8781 - val_loss: 0.2091 - val_accuracy: 0.8992
Epoch 6/20
5873/5873 [==============================] - 27s 5ms/step - loss: 0.2437 - accuracy: 0.8804 - val_loss: 0.2042 - val_accuracy: 0.8951
Epoch 7/20
5873/5873 [==============================] - 26s 5ms/step - loss: 0.2414 - accuracy: 0.8823 - val_loss: 0.2048 - val_ac

## 11. Evaluate Final Model on Test Set

We evaluate the final model using the test data and print:
- Final test accuracy,
- Full classification report,
- Confusion matrix.



In [12]:
# Evaluation
loss, acc = model.evaluate(X_test_final, y_test_final)
print(f"\nFinal Test Accuracy: {acc:.4f}")

y_pred = model.predict(X_test_final)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test_final, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_labels, y_pred_labels, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_true_labels, y_pred_labels))


1469/1469 [==============================] - 4s 3ms/step - loss: 0.1908 - accuracy: 0.8976

Final Test Accuracy: 0.8976
1469/1469 [==============================] - 2s 1ms/step

Classification Report:
              precision    recall  f1-score   support

          E1       0.91      0.98      0.94     16231
          E2       0.97      0.75      0.84     16233
          E3       0.83      0.97      0.90     14520

    accuracy                           0.90     46984
   macro avg       0.90      0.90      0.89     46984
weighted avg       0.91      0.90      0.89     46984

Confusion Matrix:
[[15979   123   129]
 [ 1446 12115  2672]
 [  205   235 14080]]


## 12. Save Final Model

We save the final trained model to disk for future inference or deployment.


In [14]:
model.save("../models/cnn1d_model_complete_dataset.h5")
print("✅ Final model saved as '../models/cnn1d_model_complete_dataset.h5'")


✅ Final model saved as 'cnn1d_best_random_model.h5'
